In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from operator import itemgetter

In [4]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it doesn't know.

Current conversation:
{message_log}

Human:
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template = TEMPLATE)

In [5]:
chat = ChatOpenAI(max_tokens = 100,
                  model_name = 'gpt-4.1-mini',
                  model_kwargs = {'seed': 365},
                  temperature = 0)

C:\Users\Korisnik\anaconda3\envs\langchain_env\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [34]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),
                                       memory_key = "message_log")

In [24]:
question = "What is the best book to read if i want to learn about economy?"
#question = "Can you give me some intresting book to read that is not too popular?"

In [25]:
dictionary_output = RunnablePassthrough.assign(message_log = RunnableLambda(chat_memory.load_memory_variables) | itemgetter('message_log')).invoke({'question': question})

In [26]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [27]:
ai_message_output = chat.invoke(prompt_value_output)

In [28]:
response = StrOutputParser().invoke(ai_message_output)

In [29]:
chat_memory.save_context(inputs = {'input': question},
                        outputs = {'output': response})

In [30]:
chat_memory.load_memory_variables({})

{'message_log': 'The human asks for an interesting book recommendation that is not too popular. The AI suggests "Stoner" by John Williams, a novel that explores themes of perseverance, love, and the passage of time with subtle depth and emotional complexity. When asked for a book recommendation on economics, the AI suggests "Economics in One Lesson" by Henry Hazlitt as a clear and accessible introduction to fundamental economic principles.'}

In [33]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables)
         | itemgetter('message_log'))
    | prompt_template
    | chat
    | StrOutputParser()
)

question = "What else can you suggest me?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question},
                         outputs = {'output': response})
response

'Sure! Here are a few more vacation spot suggestions, each with unique experiences depending on what you’re looking for:\n\n1. **Kyoto, Japan**  \n   If you love history, culture, and stunning temples, Kyoto is a fantastic choice. You can explore over 1,600 Buddhist temples, traditional tea houses, and beautiful gardens. The city is especially magical during cherry blossom season in spring or the vibrant autumn foliage.\n\n2. **Amalfi Coast, Italy**  \n   For breathtaking'

In [38]:
@chain
def memory_chain(question) :
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables)
             | itemgetter('message_log'))
        | prompt_template
        | chat
        | StrOutputParser()
    )

    chain1.get_graph().print_ascii()
    
    response = chain1.invoke({'question':question})
    
    chat_memory.save_context(inputs = {'input':question},
                             outputs = {'output': response})
    return response

In [40]:
memory_chain.invoke("Do you have some other cars to suggest me?") #Change question

              +----------------------------+          
              | Parallel<message_log>Input |          
              +----------------------------+          
                    **             ***                
                 ***                  ***             
               **                        **           
+-----------------------+                  **         
| load_memory_variables |                   *         
+-----------------------+                   *         
            *                               *         
            *                               *         
            *                               *         
        +--------+                   +-------------+  
        | Lambda |                   | Passthrough |  
        +--------+**                 +-------------+  
                    **              **                
                      ***        ***                  
                         **    **                     
          

'Absolutely! Here are a few more options you might consider within the 5,000 euros budget, typically looking at used models around 8 to 15 years old:\n\n1. **Renault Clio (2005-2010)** – A popular supermini in Europe, known for its comfortable ride and decent fuel economy. Parts and servicing are generally affordable, making it a practical choice.\n\n2. **Peugeot 207 (2006-2012)** – Stylish and comfortable, the '